In [1]:
import os.path
import keras.applications.mobilenet
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import PIL.Image
import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:
DATA_ROOT = os.path.expanduser("~/Documents/data/")
BATCH_SIZE = 8
STEPS_PER_EPOCH = 100

The model: two dense layers on top of a mobile Net

In [3]:
def new_model():
    base_model = MobileNet(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    prediction = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=prediction)

Training

In [4]:
def train(model, data_gen, directory, layer_count = None, params={},
          epochs=4, validation_directory=None, validation_generator=None):

    if layer_count == None:
        layer_count = len(model.layers)

    for layer in model.layers[: -layer_count]:
        layer.trainable = False

    for layer in model.layers[-layer_count:]:
        layer.trainable = True

    gen = data_gen.flow_from_directory(directory,
                                       class_mode='binary',
                                       batch_size=BATCH_SIZE)

    if validation_generator:
        val_gen = validation_generator.flow_from_directory(validation_directory,
                                                    class_mode='binary')
    else:
        val_gen = None

    model.compile(**params)
    model.fit_generator(gen,
                        epochs=epochs,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        validation_data=val_gen)

In [5]:
DENSE_TRAIN = {'optimizer': 'rmsprop',
               'loss': 'binary_crossentropy',
               'metrics': ['accuracy']}

In [6]:
BASE_TRAIN = {'optimizer': SGD(lr=0.0001, momentum=0.9),
              'loss': 'binary_crossentropy',
              'metrics': ['accuracy']}

In [7]:
def train1(model, data_gen, directory, epochs=4,
           validation_generator=None, validation_directory=None):
    """Train the dense layers"""
    train(model, data_gen, directory,
          epochs=epochs,
          trainable_layers=dense_layers(model),
          params=DENSE_TRAIN)

In [8]:
def train2(model, data_gen, directory, epochs=4,
           validation_generator=None, validation_directory=None):
    """Train some of the base layers"""
    train(model, data_gen, directory,
          epochs=epochs,
          trainable_layers=base_layers(model)[-16:],
          params=BASE_TRAIN)

In [9]:
model = new_model()
model.summary()
dg_train = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
dg_val = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)

/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [10]:
train(model, dg_train, os.path.join(DATA_ROOT, 'part1'),
      params=DENSE_TRAIN,
      layer_count = 2,
      validation_directory=os.path.join(DATA_ROOT, 'part2'),
      validation_generator=dg_val)
train(model, dg_train, os.path.join(DATA_ROOT, 'part1'),
      params=BASE_TRAIN,
      layer_count = 19,
      validation_generator=dg_val,
      validation_directory=os.path.join(DATA_ROOT, 'part2'))

Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/4
100/100 [==============================] - 163s 2s/step - loss: 4.2305 - acc: 0.6762 - val_loss: 0.3576 - val_acc: 0.8738
Epoch 2/4
100/100 [==============================] - 166s 2s/step - loss: 0.3313 - acc: 0.9050 - val_loss: 0.2428 - val_acc: 0.9293
Epoch 3/4
100/100 [==============================] - 168s 2s/step - loss: 0.3844 - acc: 0.8912 - val_loss: 0.0589 - val_acc: 0.9792
Epoch 4/4
100/100 [==============================] - 172s 2s/step - loss: 0.3352 - acc: 0.9038 - val_loss: 0.2859 - val_acc: 0.9097
Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Epoch 1/4
100/100 [==============================] - 179s 2s/step - loss: 0.1608 - acc: 0.9400 - val_loss: 0.0804 - val_acc: 0.9702
Epoch 2/4
100/100 [==============================] - 178s 2s/step - loss: 0.1

In [11]:
train(model, dg_train, os.path.join(DATA_ROOT, 'part1'),
      params=BASE_TRAIN,
      layer_count = 19,
      validation_generator=dg_val,
      validation_directory=os.path.join(DATA_ROOT, 'part2'))

Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Epoch 1/4
100/100 [==============================] - 179s 2s/step - loss: 0.1595 - acc: 0.9450 - val_loss: 0.0450 - val_acc: 0.9840
Epoch 2/4
 99/100 [============================>.] - ETA: 0s - loss: 0.1034 - acc: 0.9646

KeyboardInterrupt: 